In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import when
from pyspark.sql import Row, Column
from pyspark.sql.functions import initcap


Ucitavam podatke (preuzeti su sa https://github.com/fivethirtyeight/data). Radim sa podacima o NBA ligi, prvo podatke o svim utakmicama od nastanka lige, kasnije sa podacima o igracima.

In [0]:
path = "/FileStore/tables/nba-1.csv"
df = spark.read.option("inferSchema", "true").option("header", "true").csv(path, sep=',')
df.show()

+---------+------------+-----+-------+-------+----------+----------+-----------+-------+------------+---+---------+---------+---------+------+------------+-------+---------+---------+-------------+-----------+----------+-----+
|gameorder|     game_id|lg_id|_iscopy|year_id| date_game|seasongame|is_playoffs|team_id|     fran_id|pts|    elo_i|    elo_n|win_equiv|opp_id|    opp_fran|opp_pts|opp_elo_i|opp_elo_n|game_location|game_result|  forecast|notes|
+---------+------------+-----+-------+-------+----------+----------+-----------+-------+------------+---+---------+---------+---------+------+------------+-------+---------+---------+-------------+-----------+----------+-----+
|        1|194611010TRH|  NBA|      0|   1947|1946-11-01|         1|          0|    TRH|     Huskies| 66|   1300.0|1293.2767| 40.29483|   NYK|      Knicks|     68|   1300.0|1306.7233|            H|          L|0.64006501| null|
|        1|194611010TRH|  NBA|      1|   1947|1946-11-01|         1|          0|    NYK|    

Brisem kolone koje mi nece trebati

In [0]:
df = df.drop("game_id", "lg_id", "_iscopy","seasongame", "elo_i", "elo_n", "win_equiv", "opp_elo_i", "opp_elo_n","game_location", "forecast", "notes")
df.show()


+---------+-------+----------+-----------+-------+------------+---+------+------------+-------+-----------+
|gameorder|year_id| date_game|is_playoffs|team_id|     fran_id|pts|opp_id|    opp_fran|opp_pts|game_result|
+---------+-------+----------+-----------+-------+------------+---+------+------------+-------+-----------+
|        1|   1947|1946-11-01|          0|    TRH|     Huskies| 66|   NYK|      Knicks|     68|          L|
|        1|   1947|1946-11-01|          0|    NYK|      Knicks| 68|   TRH|     Huskies|     66|          W|
|        2|   1947|1946-11-02|          0|    CHS|       Stags| 63|   NYK|      Knicks|     47|          W|
|        2|   1947|1946-11-02|          0|    NYK|      Knicks| 47|   CHS|       Stags|     63|          L|
|        3|   1947|1946-11-02|          0|    DTF|     Falcons| 33|   WSC|    Capitols|     50|          L|
|        3|   1947|1946-11-02|          0|    WSC|    Capitols| 50|   DTF|     Falcons|     33|          W|
|        4|   1947|1946-11-0

Brisem suvisne redove. Svaka dva reda u ucitanom fajlu prikazuju istu utakmicu(vidi se po koloni gameorder).Meni treba samo jedan.

In [0]:
df = df.dropDuplicates(["gameorder"])
df.show()

+---------+-------+----------+-----------+-------+------------+---+------+------------+-------+-----------+
|gameorder|year_id| date_game|is_playoffs|team_id|     fran_id|pts|opp_id|    opp_fran|opp_pts|game_result|
+---------+-------+----------+-----------+-------+------------+---+------+------------+-------+-----------+
|        1|   1947|1946-11-01|          0|    TRH|     Huskies| 66|   NYK|      Knicks|     68|          L|
|        2|   1947|1946-11-02|          0|    CHS|       Stags| 63|   NYK|      Knicks|     47|          W|
|        3|   1947|1946-11-02|          0|    DTF|     Falcons| 33|   WSC|    Capitols|     50|          L|
|        4|   1947|1946-11-02|          0|    BOS|     Celtics| 53|   PRO|Steamrollers|     59|          L|
|        5|   1947|1946-11-02|          0|    PIT|     Ironmen| 51|   STB|     Bombers|     56|          L|
|        6|   1947|1946-11-03|          0|    TRH|     Huskies| 60|   CLR|      Rebels|     71|          L|
|        7|   1947|1946-11-0

Raspored utakmica mi nije potreban, posto imam datume.

In [0]:
df = df.drop("gameorder")

Menjam vredenost kolone, umesto da mi pise 0 stavljam not_playoffs ili playoffs, u slucaju da je utakmica plejofa

In [0]:

df = df.withColumn("is_playoffs", when(df.is_playoffs == 0, "not_playoffs" ).otherwise("playoffs"))
df.show(1500)

+-------+----------+------------+-------+------------+---+------+------------+-------+-----------+
|year_id| date_game| is_playoffs|team_id|     fran_id|pts|opp_id|    opp_fran|opp_pts|game_result|
+-------+----------+------------+-------+------------+---+------+------------+-------+-----------+
|   1947|1946-11-01|not_playoffs|    TRH|     Huskies| 66|   NYK|      Knicks|     68|          L|
|   1947|1946-11-02|not_playoffs|    CHS|       Stags| 63|   NYK|      Knicks|     47|          W|
|   1947|1946-11-02|not_playoffs|    DTF|     Falcons| 33|   WSC|    Capitols|     50|          L|
|   1947|1946-11-02|not_playoffs|    BOS|     Celtics| 53|   PRO|Steamrollers|     59|          L|
|   1947|1946-11-02|not_playoffs|    PIT|     Ironmen| 51|   STB|     Bombers|     56|          L|
|   1947|1946-11-03|not_playoffs|    TRH|     Huskies| 60|   CLR|      Rebels|     71|          L|
|   1947|1946-11-04|not_playoffs|    PIT|     Ironmen| 56|   WSC|    Capitols|     71|          L|
|   1947|1

Menjam nazive kolona

In [0]:
df = df.withColumnRenamed("year_id", "Season")\
        .withColumnRenamed("team_id", "Home_team")\
        .withColumnRenamed("fran_id", "Franchise_home")\
        .withColumnRenamed("pts", "Home_pts")\
        .withColumnRenamed("opp_id", "Opponent_team")\
        .withColumnRenamed("opp_fran", "Franchise_opponent")\
        .withColumnRenamed("opp_pts", "Opponent_pts")\
        .withColumnRenamed("game_result", "Game_result")

df.show()


+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+
|Season| date_game| is_playoffs|Home_team|Franchise_home|Home_pts|Opponent_team|Franchise_opponent|Opponent_pts|Game_result|
+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+
|  1947|1946-11-01|not_playoffs|      TRH|       Huskies|      66|          NYK|            Knicks|          68|          L|
|  1947|1946-11-02|not_playoffs|      CHS|         Stags|      63|          NYK|            Knicks|          47|          W|
|  1947|1946-11-02|not_playoffs|      DTF|       Falcons|      33|          WSC|          Capitols|          50|          L|
|  1947|1946-11-02|not_playoffs|      BOS|       Celtics|      53|          PRO|      Steamrollers|          59|          L|
|  1947|1946-11-02|not_playoffs|      PIT|       Ironmen|      51|          STB|           Bombers|          56|          L|


Dodajem novu kolonu koja predstavlja ukupan broj poena na utakmici

In [0]:
df = df.withColumn("Total_pts", col("Home_pts") + col("Opponent_pts"))
df.show()

+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|Season| date_game| is_playoffs|Home_team|Franchise_home|Home_pts|Opponent_team|Franchise_opponent|Opponent_pts|Game_result|Total_pts|
+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|  1947|1946-11-01|not_playoffs|      TRH|       Huskies|      66|          NYK|            Knicks|          68|          L|      134|
|  1947|1946-11-02|not_playoffs|      DTF|       Falcons|      33|          WSC|          Capitols|          50|          L|       83|
|  1947|1946-11-03|not_playoffs|      TRH|       Huskies|      60|          CLR|            Rebels|          71|          L|      131|
|  1947|1946-11-07|not_playoffs|      STB|       Bombers|      63|          NYK|            Knicks|          68|          L|      131|
|  1947|1946-11-08|not_playoffs|      TRH|       Huskie

Podesavam da mi svi nazivi kolona budu malim slovima, zbog lepseg izgleda

In [0]:
df = df.select([col(x).alias(x.lower()) for x in df.columns])
df.show()

+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|season| date_game| is_playoffs|home_team|franchise_home|home_pts|opponent_team|franchise_opponent|opponent_pts|game_result|total_pts|
+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|  1947|1946-11-01|not_playoffs|      TRH|       Huskies|      66|          NYK|            Knicks|          68|          L|      134|
|  1947|1946-11-02|not_playoffs|      DTF|       Falcons|      33|          WSC|          Capitols|          50|          L|       83|
|  1947|1946-11-03|not_playoffs|      TRH|       Huskies|      60|          CLR|            Rebels|          71|          L|      131|
|  1947|1946-11-07|not_playoffs|      STB|       Bombers|      63|          NYK|            Knicks|          68|          L|      131|
|  1947|1946-11-08|not_playoffs|      TRH|       Huskie

Posto su sva polja u napisana sa prvim velikim slovom, namestam da mi i kolona is_playoffs tako izgleda

In [0]:
df = df.withColumn("is_playoffs", initcap(df["is_playoffs"]))
df.show()

+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|season| date_game| is_playoffs|home_team|franchise_home|home_pts|opponent_team|franchise_opponent|opponent_pts|game_result|total_pts|
+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|  1947|1946-11-01|Not_playoffs|      TRH|       Huskies|      66|          NYK|            Knicks|          68|          L|      134|
|  1947|1946-11-02|Not_playoffs|      DTF|       Falcons|      33|          WSC|          Capitols|          50|          L|       83|
|  1947|1946-11-03|Not_playoffs|      TRH|       Huskies|      60|          CLR|            Rebels|          71|          L|      131|
|  1947|1946-11-07|Not_playoffs|      STB|       Bombers|      63|          NYK|            Knicks|          68|          L|      131|
|  1947|1946-11-08|Not_playoffs|      TRH|       Huskie

Pravim putanju gde ce se sacuvati podaci i pod kojim imenom

In [0]:
path = 'dbfs:/ispit/'
csv_name = path + 'df.csv'
parquet_name = path + 'df.parquet'
orc_name = path + 'df.orc'


df.write.csv(csv_name, header=True)
df_csv = spark.read.csv(csv_name, header=True)
df_csv.printSchema()
df_csv = spark.read.csv(csv_name, header=True, inferSchema=True)
df_csv.printSchema()


root
 |-- season: string (nullable = true)
 |-- date_game: string (nullable = true)
 |-- is_playoffs: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- franchise_home: string (nullable = true)
 |-- home_pts: string (nullable = true)
 |-- opponent_team: string (nullable = true)
 |-- franchise_opponent: string (nullable = true)
 |-- opponent_pts: string (nullable = true)
 |-- game_result: string (nullable = true)
 |-- total_pts: string (nullable = true)

root
 |-- season: integer (nullable = true)
 |-- date_game: date (nullable = true)
 |-- is_playoffs: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- franchise_home: string (nullable = true)
 |-- home_pts: integer (nullable = true)
 |-- opponent_team: string (nullable = true)
 |-- franchise_opponent: string (nullable = true)
 |-- opponent_pts: integer (nullable = true)
 |-- game_result: string (nullable = true)
 |-- total_pts: integer (nullable = true)



In [0]:
df.write.save(parquet_name, format='parquet') 
spark.read.load(parquet_name, format='parquet').show() 

+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|season| date_game| is_playoffs|home_team|franchise_home|home_pts|opponent_team|franchise_opponent|opponent_pts|game_result|total_pts|
+------+----------+------------+---------+--------------+--------+-------------+------------------+------------+-----------+---------+
|  1947|1946-11-02|Not_playoffs|      CHS|         Stags|      63|          NYK|            Knicks|          47|          W|      110|
|  1947|1946-11-04|Not_playoffs|      PIT|       Ironmen|      56|          WSC|          Capitols|          71|          L|      127|
|  1947|1946-11-07|Not_playoffs|      PIT|       Ironmen|      75|          PHW|          Warriors|          81|          L|      156|
|  1947|1946-11-07|Not_playoffs|      PRO|  Steamrollers|      73|          CHS|             Stags|          65|          W|      138|
|  1947|1946-11-09|Not_playoffs|      TRH|       Huskie

Ucitavam drugi deo podataka sa istog sajta i sredjujem podatke slicno kao i prethodne

In [0]:
path = "/FileStore/tables/players.csv"
df1 = spark.read.option("inferSchema", "true").option("header", "true").csv(path, sep=',')
df1.show()

+-------------------+---------+------+----+----+--------------+--------------+------------+------------+--------------+------------+----------------+----------------+--------------+------------+
|        player_name|player_id|season|poss|  mp|raptor_offense|raptor_defense|raptor_total|   war_total|war_reg_season|war_playoffs|predator_offense|predator_defense|predator_total| pace_impact|
+-------------------+---------+------+----+----+--------------+--------------+------------+------------+--------------+------------+----------------+----------------+--------------+------------+
|     Alaa Abdelnaby|abdelal01|  1991| 640| 303|  -3.938449568|  -0.510076405|-4.448525973|-0.265191008|  -0.226162818| -0.03902819|    -3.905731828|    -1.696796053|  -5.602527881| 0.199240629|
|     Alaa Abdelnaby|abdelal01|  1992|1998| 959|  -2.553849264|  -0.197943029|-2.751792294|-0.006892965|   0.070321631|-0.077214596|    -2.499575947|    -0.721876322|  -3.221452269|  0.48543621|
|     Alaa Abdelnaby|abde

In [0]:
df1 = df1.drop("player_id", "war_reg_season", "war_playoffs","predator_offense","raptor_total","war_total", "predator_defense", "predator_total")
df1.show()

+-------------------+------+----+----+--------------+--------------+------------+
|        player_name|season|poss|  mp|raptor_offense|raptor_defense| pace_impact|
+-------------------+------+----+----+--------------+--------------+------------+
|     Alaa Abdelnaby|  1991| 640| 303|  -3.938449568|  -0.510076405| 0.199240629|
|     Alaa Abdelnaby|  1992|1998| 959|  -2.553849264|  -0.197943029|  0.48543621|
|     Alaa Abdelnaby|  1993|2754|1379|  -2.373735823|  -2.069807932| 0.488265925|
|     Alaa Abdelnaby|  1994| 320| 159|  -6.140056457|  -2.748312039| 0.511152215|
|     Alaa Abdelnaby|  1995| 984| 506|   -3.84654328|  -1.268012224| 1.545921938|
|Kareem Abdul-Jabbar|  1977|7674|3483|   4.544044142|   3.103854696| -0.50296209|
|Kareem Abdul-Jabbar|  1978|5343|2399|   3.447574132|   3.423490189|-0.566163079|
|Kareem Abdul-Jabbar|  1979|7826|3524|   2.950434641|   2.534174878|-0.799426048|
|Kareem Abdul-Jabbar|  1980|8226|3761|   3.121840014|   2.430630696|-0.726437248|
|Kareem Abdul-Ja

Radicu analilzu samo odredjenih igraca, a ih odmah izdvajam

In [0]:
df1 = df1.filter((df1.player_name == 'Michael Jordan') | (df1.player_name == 'LeBron James') )
df1.show(50)

+--------------+------+----+----+-----------------+------------------+------------------+
|   player_name|season|poss|  mp|   raptor_offense|    raptor_defense|       pace_impact|
+--------------+------+----+----+-----------------+------------------+------------------+
|  LeBron James|  2004|5974|3122|      1.337109873|      -0.298264024|       -0.31800111|
|  LeBron James|  2005|6404|3388|      5.094678357|       1.423964185|      -0.407580049|
|  LeBron James|  2006|7452|3965|      6.415884113|       0.840637535|      -0.123552575|
|  LeBron James|  2007|7717|4083|      4.632207144|       2.641377662|      -0.138090935|
|  LeBron James|  2008|6753|3579|         6.802467|       1.626495475|       0.015236558|
|  LeBron James|  2009|6760|3634|      9.334993762|       3.245820847|      -0.344555623|
|  LeBron James|  2010|6611|3426|      9.196731423|       2.250083757|      -0.376950737|
|  LeBron James|  2011|7544|3985|      6.041355599|       1.546103098|      -0.219552029|
|  LeBron 

In [0]:
df1 = df1.withColumnRenamed("poss", "Possessions")\
        .withColumnRenamed("mp", "Minutes")\
        .withColumnRenamed("raptor_offense", "Offense")\
        .withColumnRenamed("raptor_defense", "Defense")\
        .withColumnRenamed("pace_impact", "Impact")
df1.show()

+--------------+------+-----------+-------+-----------------+------------------+------------------+
|   player_name|season|Possessions|Minutes|          Offense|           Defense|            Impact|
+--------------+------+-----------+-------+-----------------+------------------+------------------+
|  LeBron James|  2004|       5974|   3122|      1.337109873|      -0.298264024|       -0.31800111|
|  LeBron James|  2005|       6404|   3388|      5.094678357|       1.423964185|      -0.407580049|
|  LeBron James|  2006|       7452|   3965|      6.415884113|       0.840637535|      -0.123552575|
|  LeBron James|  2007|       7717|   4083|      4.632207144|       2.641377662|      -0.138090935|
|  LeBron James|  2008|       6753|   3579|         6.802467|       1.626495475|       0.015236558|
|  LeBron James|  2009|       6760|   3634|      9.334993762|       3.245820847|      -0.344555623|
|  LeBron James|  2010|       6611|   3426|      9.196731423|       2.250083757|      -0.376950737|


In [0]:
df1 = df1.select([col(x).alias(x.lower()) for x in df1.columns])
df1.show()

+--------------+------+-----------+-------+-----------------+------------------+------------------+
|   player_name|season|possessions|minutes|          offense|           defense|            impact|
+--------------+------+-----------+-------+-----------------+------------------+------------------+
|  LeBron James|  2004|       5974|   3122|      1.337109873|      -0.298264024|       -0.31800111|
|  LeBron James|  2005|       6404|   3388|      5.094678357|       1.423964185|      -0.407580049|
|  LeBron James|  2006|       7452|   3965|      6.415884113|       0.840637535|      -0.123552575|
|  LeBron James|  2007|       7717|   4083|      4.632207144|       2.641377662|      -0.138090935|
|  LeBron James|  2008|       6753|   3579|         6.802467|       1.626495475|       0.015236558|
|  LeBron James|  2009|       6760|   3634|      9.334993762|       3.245820847|      -0.344555623|
|  LeBron James|  2010|       6611|   3426|      9.196731423|       2.250083757|      -0.376950737|


Razdvajam vrednosti iz jedne kolone u dve

In [0]:
df1 = df1.select(split(df1['player_name'], ' ')[0].alias('first_name'), 
               split(df1['player_name'], ' ')[1].alias('last_name'), "season", "possessions", "minutes", "offense", "defense", "impact")
df1.show()

+----------+---------+------+-----------+-------+-----------------+------------------+------------------+
|first_name|last_name|season|possessions|minutes|          offense|           defense|            impact|
+----------+---------+------+-----------+-------+-----------------+------------------+------------------+
|    LeBron|    James|  2004|       5974|   3122|      1.337109873|      -0.298264024|       -0.31800111|
|    LeBron|    James|  2005|       6404|   3388|      5.094678357|       1.423964185|      -0.407580049|
|    LeBron|    James|  2006|       7452|   3965|      6.415884113|       0.840637535|      -0.123552575|
|    LeBron|    James|  2007|       7717|   4083|      4.632207144|       2.641377662|      -0.138090935|
|    LeBron|    James|  2008|       6753|   3579|         6.802467|       1.626495475|       0.015236558|
|    LeBron|    James|  2009|       6760|   3634|      9.334993762|       3.245820847|      -0.344555623|
|    LeBron|    James|  2010|       6611|   34

In [0]:
df1.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- possessions: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- offense: double (nullable = true)
 |-- defense: double (nullable = true)
 |-- impact: double (nullable = true)



Zaokruzujem vrednosti na dve decimale

In [0]:
df1 = df1.withColumn("offense", round(df1["offense"], 2))\
            .withColumn("defense", round(df1["defense"], 2))\
            .withColumn("impact", round(df1["impact"], 2))

df1.show()

+----------+---------+------+-----------+-------+-------+-------+------+
|first_name|last_name|season|possessions|minutes|offense|defense|impact|
+----------+---------+------+-----------+-------+-------+-------+------+
|    LeBron|    James|  2004|       5974|   3122|   1.34|   -0.3| -0.32|
|    LeBron|    James|  2005|       6404|   3388|   5.09|   1.42| -0.41|
|    LeBron|    James|  2006|       7452|   3965|   6.42|   0.84| -0.12|
|    LeBron|    James|  2007|       7717|   4083|   4.63|   2.64| -0.14|
|    LeBron|    James|  2008|       6753|   3579|    6.8|   1.63|  0.02|
|    LeBron|    James|  2009|       6760|   3634|   9.33|   3.25| -0.34|
|    LeBron|    James|  2010|       6611|   3426|    9.2|   2.25| -0.38|
|    LeBron|    James|  2011|       7544|   3985|   6.04|   1.55| -0.22|
|    LeBron|    James|  2012|       6293|   3309|    7.9|   2.09| -0.36|
|    LeBron|    James|  2013|       7265|   3837|   8.53|   1.09| -0.61|
|    LeBron|    James|  2014|       7021|   3665|  

Upisujem podatke isto kao i prethodne

In [0]:
path = 'dbfs:/ispit_bd/'
csv_name = path + 'df1.csv'
parquet_name = path + 'df1.parquet'
orc_name = path + 'df1.orc'


df1.write.csv(csv_name, header=True)
df1_csv = spark.read.csv(csv_name, header=True)
df1_csv.printSchema()
df1_csv = spark.read.csv(csv_name, header=True, inferSchema=True)
df1_csv.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- season: string (nullable = true)
 |-- possessions: string (nullable = true)
 |-- minutes: string (nullable = true)
 |-- offense: string (nullable = true)
 |-- defense: string (nullable = true)
 |-- impact: string (nullable = true)

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- possessions: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- offense: double (nullable = true)
 |-- defense: double (nullable = true)
 |-- impact: double (nullable = true)



In [0]:
df1.write.save(parquet_name, format='parquet') 
spark.read.load(parquet_name, format='parquet').show() 

+----------+---------+------+-----------+-------+-------+-------+------+
|first_name|last_name|season|possessions|minutes|offense|defense|impact|
+----------+---------+------+-----------+-------+-------+-------+------+
|    LeBron|    James|  2004|       5974|   3122|   1.34|   -0.3| -0.32|
|    LeBron|    James|  2005|       6404|   3388|   5.09|   1.42| -0.41|
|    LeBron|    James|  2006|       7452|   3965|   6.42|   0.84| -0.12|
|    LeBron|    James|  2007|       7717|   4083|   4.63|   2.64| -0.14|
|    LeBron|    James|  2008|       6753|   3579|    6.8|   1.63|  0.02|
|    LeBron|    James|  2009|       6760|   3634|   9.33|   3.25| -0.34|
|    LeBron|    James|  2010|       6611|   3426|    9.2|   2.25| -0.38|
|    LeBron|    James|  2011|       7544|   3985|   6.04|   1.55| -0.22|
|    LeBron|    James|  2012|       6293|   3309|    7.9|   2.09| -0.36|
|    LeBron|    James|  2013|       7265|   3837|   8.53|   1.09| -0.61|
|    LeBron|    James|  2014|       7021|   3665|  